Computation Pipeline to:
- Pre-process single cell RNA-seq dataset using Scanpy
- Apply denoising on single cell RNA-seq data using MAGIC
- Compute Reaction Activity Score
- Compute single-cell Flux Balance Analysis
- Perform Flux Cluster Analysis

Load libraries

In [ ]:
import pandas as pd
import numpy as np
from scanpy import AnnData
import scanpy as sc
import cobra as cb
import numpy as np
import time
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.sparse import csr_matrix
import sys

Set denoising strategy. Three possible denoisers can be used (magic,enhance, saver). Denoising can be applied on the readcounts or on the RAS matrix.

In [ ]:
denoising_strategy="magic_counts" 
compute_fluxes=True
save_flux=True
save_ras=True
save_counts=False

In [ ]:
if denoising_strategy=="epsilon":
    eps=0.01
else:
    eps=0

Set name of the dataset

In [ ]:
#file_input="datasetGSE110949"
file_input="datasetE-GEOD-86618"
#file_input="datasetGSE118056"

## Load RNA-seq and pre-processing before RAS computation

Load single-cell dataset and stored in AnnData object

In [ ]:
adata=sc.read_h5ad("data/"+file_input)
adata

Quality check filtering (remove cell with few expressed genes and genes not expressed in 3 cells at least)

In [ ]:
sc.pp.filter_cells(adata, min_genes=2000)
sc.pp.filter_genes(adata, min_cells=3)
adata

TPM normalization

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e6)

Store cell labels 

In [ ]:
cells=list(adata.obs.index)

## Application of denoising

Before denoising

In [ ]:
adata.to_df()

Applying denoising on the readcounts matrix. 
- MAGIC can be used using the Scanpy function sc.external.pp.magic
- To use ENHANCE, you need to download the code from the ENHANCE repository https://github.com/yanailab/enhance
- SAVER denoiser is available as R-package. We provide an R-script to run SAVER named saver_readcounts.R

In [ ]:
if denoising_strategy=="magic_counts":
    sc.external.pp.magic(adata,random_state=0)   
adata

After denoising

In [ ]:
adata.to_df()

Save read counts data

In [ ]:
if save_counts:
    adata.X = csr_matrix(adata.X)
    adata.write("outputs/readcounts/"+file_input)

## Load metabolic model

Read the SBML model

In [ ]:
if file_input=="datasetE-GEOD-86618":
    model=cb.io.read_sbml_model('models/ENGRO2_ensg.xml')       #gene ensg notation
else:
    model=cb.io.read_sbml_model('models/ENGRO2_genesymbol.xml')       #gene ensg notation
model

In [ ]:
reactions=[reaction.id for reaction in model.reactions]

Set objective function

In [ ]:
if file_input=="datasetE-GEOD-86618":
    model.objective="ATPM"
else:
    model.objective="Biomass"

## FVA computation

In [ ]:
from cobra.flux_analysis import flux_variability_analysis

In [ ]:
#dfFVA=flux_variability_analysis(model,fraction_of_optimum=0)
#dfFVA=dfFVA.round(decimals=4)
#dfFVA["maxABS"]=dfFVA.abs().max(1)
#dfFVA.to_csv("data/FVA.csv")
#dfFVA

In [ ]:
dfFVA=pd.read_csv("data/FVA.csv",index_col=0)

In [ ]:
dfFVA

## RAS computation 

Compute Reaction Activity Scores

In [ ]:
from utils import RAS_computation as rc

In [ ]:
adata.to_df()

In [ ]:
ras_object=rc(adata,model)
ras_adata=ras_object.compute()
ras_adata

The resulting ras_adata is an anndata object where the feature now are the RAS

In [ ]:
ras_adata.to_df()

Applying denoising on the RAS matrix

In [ ]:
if denoising_strategy=="magic_ras":
    sc.external.pp.magic(ras_adata,random_state=0)

After denoising

In [ ]:
ras_adata.to_df()

Save RAS data

In [ ]:
if save_ras:
    ras_adata.X = csr_matrix(ras_adata.X)
    ras_adata.write("outputs/"+file_input+"_"+denoising_strategy)

## Single cell Flux Balance analysis

In [ ]:
from utils import scFBA

Compute single cell FBA

In [ ]:
if compute_fluxes:
    flux_adata=scFBA(model,ras_adata,dfFVA,eps=eps,verbose=False)

Save results of flux data

In [ ]:
if save_flux and save_flux:
    flux_adata.write("outputs/fluxes/"+file_input+"_"+denoising_strategy)